In [1]:
import qiskit
from qiskit import QuantumCircuit as QiskitCircuit
from qiskit.quantum_info import Operator as QiskitOperator
import cirq
from cirq import Circuit as CirqCircuit
import braket
from braket.circuits import Circuit as BraketCircuit
from braket.circuits.gate import Gate as BraketGate
from braket.circuits.instruction import Instruction as BraketInstruction
from braket.circuits.unitary_calculation import calculate_unitary
from qbraid.transpiler.transpiler import qbraid_wrapper
from qbraid.transpiler2.transpiler import qbraid_wrapper as qbraid_wrapper2
import numpy as np
from typing import List

### Shared gate circuit functions and unitary helper function

In [2]:
def qiskit_shared_gates_circuit():
    """Returns qiskit `QuantumCircuit` for qBraid `TestSharedGates`."""
    
    qiskit_circuit = QiskitCircuit(4)

    qiskit_circuit.h([0, 1, 2, 3])
    qiskit_circuit.x([0, 1])
    qiskit_circuit.y(2)
    qiskit_circuit.z(3)
    qiskit_circuit.s(0)
    qiskit_circuit.sdg(1)
    qiskit_circuit.t(2)
    qiskit_circuit.tdg(3)
    qiskit_circuit.rx(np.pi/4, 0)
    qiskit_circuit.ry(np.pi/2, 1)
    qiskit_circuit.rz(3*np.pi/4, 2)
    qiskit_circuit.p(np.pi/8, 3)
    qiskit_circuit.sx(0)
    qiskit_circuit.sxdg(1)
    qiskit_circuit.iswap(2, 3)
    qiskit_circuit.swap([0, 1], [2, 3])
    qiskit_circuit.cx(0, 1)
    qiskit_circuit.cp(np.pi/4, 2, 3)

    return qiskit_circuit

In [3]:
def cirq_shared_gates_circuit(rev_qubits=False):
    """Returns cirq `Circuit` for qBraid `TestSharedGates` 
    rev_qubits=True reverses ordering of qubits."""
    
    cirq_circuit = CirqCircuit()
    qubits = [cirq.LineQubit(i) for i in range(4)]
    q3, q2, q1, q0 = qubits if rev_qubits else list(reversed(qubits))
        
    cirq_gates = [
        cirq.H(q0),
        cirq.H(q1),
        cirq.H(q2),
        cirq.H(q3),
        cirq.X(q0),
        cirq.X(q1),
        cirq.Y(q2),
        cirq.Z(q3),
        cirq.S(q0),
        cirq.ZPowGate(exponent=-0.5)(q1),
        cirq.T(q2),
        cirq.ZPowGate(exponent=-0.25)(q3),
        cirq.Rx(rads=np.pi / 4)(q0),
        cirq.Ry(rads=np.pi / 2)(q1),
        cirq.Rz(rads=3 * np.pi / 4)(q2),
        cirq.ZPowGate(exponent=1 / 8)(q3),
        cirq.XPowGate(exponent=0.5)(q0),
        cirq.XPowGate(exponent=-0.5)(q1),
        cirq.ISWAP(q2, q3),
        cirq.SWAP(q0, q2),
        cirq.SWAP(q1, q3),
        cirq.CNOT(q0, q1),
        cirq.CZPowGate(exponent=0.25)(q2, q3),
    ]
    
    for gate in cirq_gates:
        cirq_circuit.append(gate)
    
    return cirq_circuit 

In [4]:
def braket_shared_gates_instructions(get_circuit=False):
    """Returns braket `List[Instruction]` for qBraid `TestSharedGates`
    if get_circuit=True, instead returns braket `Circuit`."""

    instructions = [
        BraketInstruction(BraketGate.H(), 0),
        BraketInstruction(BraketGate.H(), 1),
        BraketInstruction(BraketGate.H(), 2),
        BraketInstruction(BraketGate.H(), 3),
        BraketInstruction(BraketGate.X(), 0),
        BraketInstruction(BraketGate.X(), 1),
        BraketInstruction(BraketGate.Y(), 2),
        BraketInstruction(BraketGate.Z(), 3),
        BraketInstruction(BraketGate.S(), 0),
        BraketInstruction(BraketGate.Si(), 1),
        BraketInstruction(BraketGate.T(), 2),
        BraketInstruction(BraketGate.Ti(), 3),
        BraketInstruction(BraketGate.Rx(np.pi/4), 0),
        BraketInstruction(BraketGate.Ry(np.pi/2), 1),
        BraketInstruction(BraketGate.Rz(3*np.pi/4), 2),
        BraketInstruction(BraketGate.PhaseShift(np.pi/8), 3),
        BraketInstruction(BraketGate.V(), 0),
        BraketInstruction(BraketGate.Vi(), 1),
        BraketInstruction(BraketGate.ISwap(), [2, 3]),
        BraketInstruction(BraketGate.Swap(), [0, 2]),
        BraketInstruction(BraketGate.Swap(), [1, 3]),
        BraketInstruction(BraketGate.CNot(), [0, 1]),
        BraketInstruction(BraketGate.CPhaseShift(np.pi/4), [2, 3]),
    ]
    
    if not get_circuit:
        return instructions
    
    braket_circuit = BraketCircuit()

    for inst in instructions:
        braket_circuit.add_instruction(inst)
    
    return braket_circuit

In [5]:
def to_unitary(circuit, qubit_count=None):
    """Calculate unitary of braket, cirq, or qiskit circuit.
    Args:
        circuit (braket Iterable[Instruction] or cirq Circuit or qiskit Circuit): The circuit specification 
            for which the unitary matrix will be calculated. 
        qubit_count (int): Number of qubits in circuit, only required for braket calculation.
    Returns:
        numpy.ndarray: A numpy array representing the `circuit` as a unitary
    """
    if isinstance(circuit, List) and isinstance(circuit[0], BraketInstruction):
        if qubit_count is None:
            raise ValueError("Must specify qubit_count.")
        return calculate_unitary(qubit_count, circuit)
    elif isinstance(circuit, CirqCircuit):
        return circuit.unitary()
    elif isinstance(circuit, QiskitCircuit):
        return QiskitOperator(circuit).data
    else:
        raise TypeError(f"to_unitary calculation not supported for type {type(circuit)}")

### Define circuits and unitaries

In [6]:
braket_instructions = braket_shared_gates_instructions()
cirq_circuit = cirq_shared_gates_circuit()
cirq_rev_circuit = cirq_shared_gates_circuit(rev_qubits=True)
qiskit_circuit = qiskit_shared_gates_circuit()

In [7]:
braket_unitary = to_unitary(braket_instructions, qubit_count=4)
cirq_unitary = to_unitary(cirq_circuit)
cirq_rev_unitary = to_unitary(cirq_rev_circuit)
qiskit_unitary = to_unitary(qiskit_circuit)

### Braket to Cirq

### Braket to Qiskit

### Cirq to Qiskit

### Cirq to Braket

### Qiskit to Braket

### Qiskit to Cirq